In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import Literal
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

@tool
def get_market_value(player_name: str):
    """Fake tool to simulate querying a database for a player's market value."""
    fake_db = {
        "Lionel Messi": "50 Mio. Euro",
        "Cristiano Ronaldo": "40 Mio. Euro",
    }
    return fake_db.get(player_name, "Market value information not available.")

tools = [get_market_value]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

def call_model(state: MessagesState):
    messages = state['messages']
    system_message = SystemMessage(content="""You are an agent tasked with determining the market value of a player.
    If the market value is mentioned, return it in the format 'XX Mio. Euro'. Otherwise, return 'Market value information not available.'""")
    response = model.invoke([system_message] + messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

market_value_graph = StateGraph(MessagesState)
market_value_graph.add_node("call_model", call_model)
market_value_graph.add_node("tools", ToolNode(tools))
market_value_graph.add_edge(START, "call_model")
market_value_graph.add_conditional_edges("call_model", should_continue)
market_value_graph.add_edge("tools", "call_model")

market_value_researcher_agent = market_value_graph.compile()


In [3]:
market_value_researcher_agent.invoke({"messages": HumanMessage(content="Lionel Messi goes to Real Madrid in 2025")})

{'messages': [HumanMessage(content='Lionel Messi goes to Real Madrid in 2025', additional_kwargs={}, response_metadata={}, id='86f37b72-7319-4742-a19a-a407189341b1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TlXvYitvihVT9aZZXuyZjM6j', 'function': {'arguments': '{"player_name":"Lionel Messi"}', 'name': 'get_market_value'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 106, 'total_tokens': 124, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cfb95ac1-1bf8-4079-acd1-252ad70ed024-0', tool_calls=[{'name': 'get_market_value', 'args': {'player_name': 'Lionel Messi'}, 'id': 'call_TlXvYitvihVT9aZZXuyZjM6j'

In [4]:
@tool
def get_current_club(player_name: str):
    """Fake tool to simulate querying a database for a player's current club."""
    fake_db = {
        "Lionel Messi": "Paris Saint-Germain",
        "Cristiano Ronaldo": "Al Nassr FC",
    }
    return fake_db.get(player_name, "Current club information not available.")

tools = [get_current_club]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

def call_model_current_club(state: MessagesState):
    messages = state['messages']
    system_message = SystemMessage(content="""You are an agent tasked with determining the current club of a player.
    If the current club is mentioned, return it. Otherwise, return 'Current club information not available.'""")
    response = model.invoke([system_message] + messages)
    return {"messages": [response]}


current_club_graph = StateGraph(MessagesState)
current_club_graph.add_node("call_model_current_club", call_model_current_club)
current_club_graph.add_node("tools", ToolNode(tools))
current_club_graph.add_edge(START, "call_model_current_club")
current_club_graph.add_conditional_edges("call_model_current_club", should_continue)
current_club_graph.add_edge("tools", "call_model_current_club")

current_club_researcher_agent = current_club_graph.compile()

In [5]:
initial_state = {"messages": [HumanMessage(content="Lionel Messi joins a new club in Paris")]}
current_club_researcher_agent.invoke(initial_state)

{'messages': [HumanMessage(content='Lionel Messi joins a new club in Paris', additional_kwargs={}, response_metadata={}, id='df57ebe9-9615-45e7-860d-b8c9ab1687a5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LrvJN5MLy0pM2lMQwHYMOLmD', 'function': {'arguments': '{"player_name":"Lionel Messi"}', 'name': 'get_current_club'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 97, 'total_tokens': 116, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-45058a1c-ee69-4f6e-8ffd-a9bcc2ec2ebe-0', tool_calls=[{'name': 'get_current_club', 'args': {'player_name': 'Lionel Messi'}, 'id': 'call_LrvJN5MLy0pM2lMQwHYMOLmD', '

In [6]:
@tool
def count_words(text: str) -> int:
    """Tool to determine the length of a text"""
    words = text.split()
    return len(words)

tools = [count_words]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

def call_model_word_count(state: MessagesState):
    messages = state['messages']
    system_message = SystemMessage(content="""You are an agent tasked with rewriting the document to ensure it contains at least 50 words.
    If the document contains fewer than 50 words, expand it too >50 words without changing the context of the text.
    Only provide the new, expanded text in your response, without explaining what changes were made.""")
    response = model.invoke([system_message] + messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

word_count_graph = StateGraph(MessagesState)
word_count_graph.add_node("call_model_word_count", call_model_word_count)
word_count_graph.add_node("tools", ToolNode(tools))
word_count_graph.add_edge(START, "call_model_word_count")
word_count_graph.add_conditional_edges("call_model_word_count", should_continue)
word_count_graph.add_edge("tools", "call_model_word_count")

word_count_rewriter_agent = word_count_graph.compile()

In [7]:
initial_state = {"messages": [HumanMessage(content="Messi will switch to Real Madrid in 2025")]}
word_count_rewriter_agent.invoke(initial_state)

{'messages': [HumanMessage(content='Messi will switch to Real Madrid in 2025', additional_kwargs={}, response_metadata={}, id='fb84a4e2-1671-4d2e-bf56-fd382c1907d8'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f8hWAafdSHDt8qk7JaISJaRn', 'function': {'arguments': '{"text":"Messi will switch to Real Madrid in 2025"}', 'name': 'count_words'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 120, 'total_tokens': 144, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0cd60dae-b34e-449e-9c05-3903f649e23d-0', tool_calls=[{'name': 'count_words', 'args': {'text': 'Messi will switch to Real Madrid in 2025'}, 'id':

### Supervisor Agent

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class ArticlePostabilityGrader(BaseModel):
    """Binary scores for verifying if an article mentions market value, current club, and meets the minimum word count of 100 words."""

    mentions_market_value: str = Field(
        description="The article mentions the player's market value, 'yes' or 'no'"
    )
    mentions_current_club: str = Field(
        description="The article mentions the player's current club, 'yes' or 'no'"
    )
    meets_100_words: str = Field(
        description="The article has at least 100 words, 'yes' or 'no'"
    )

llm_postability = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_postability_grader = llm_postability.with_structured_output(
    ArticlePostabilityGrader
)

postability_system = """You are a grader assessing whether a news article meets the following criteria:
1. Mentions the player's market value.
2. Mentions the player's current club.
3. Contains at least 100 words.
Provide three binary scores: one indicating if the article mentions the player's market value ('yes' or 'no'), one for mentioning the player's current club ('yes' or 'no'), and one for meeting the 100-word count ('yes' or 'no')."""
postability_grade_prompt = ChatPromptTemplate.from_messages(
    [("system", postability_system), ("human", "News Article:\n\n{article}")]
)

news_chef = postability_grade_prompt | structured_llm_postability_grader

In [12]:
news_chef.invoke({"article": "Messi will switch from FC Barcelona to Real Madrid in 2025"})

ArticlePostabilityGrader(mentions_market_value='no', mentions_current_club='yes', meets_100_words='no')

### Workflow

In [ ]:
class SharedArticleState(TypedDict):
    messages: List[BaseMessage]
    mentions_market_value: str
    mentions_current_club: str
    meets_100_words: str

# Normal node: Updates the state with classification results from news_chef
def update_article_state(state: SharedArticleState) -> SharedArticleState:
    article_content = state["messages"][0].content  # Extract article content from messages
    result = news_chef.invoke({"article": article_content})

    state["mentions_market_value"] = result.mentions_market_value
    state["mentions_current_club"] = result.mentions_current_club
    state["meets_100_words"] = result.meets_100_words
    return state

# Router function: Decides the next agent based on updated state
def news_chef_decider(state: SharedArticleState) -> Literal["word_count_rewriter", "current_club_researcher", "market_value_researcher"]:
    if state["mentions_market_value"] == "yes" and state["mentions_current_club"] == "yes" and state["meets_100_words"] == "yes":
        return "market_value_researcher"
    elif state["meets_100_words"] == "no":
        return "word_count_rewriter"
    else:
        return "current_club_researcher"

# Workflow setup
workflow = StateGraph(SharedArticleState)

# Add nodes for each step
workflow.add_node("news_chef", update_article_state)
workflow.add_node("word_count_rewriter", word_count_rewriter_agent)
workflow.add_node("current_club_researcher", current_club_researcher_agent)
workflow.add_node("market_value_researcher", market_value_researcher_agent)

# Set the entry point to start with news_chef
workflow.set_entry_point("news_chef")

# Conditional routing based on the router function's decision
workflow.add_conditional_edges(
    "news_chef",
    news_chef_decider,
    {
        "word_count_rewriter": "word_count_rewriter",
        "current_club_researcher": "current_club_researcher",
        "market_value_researcher": "news_chef",  # Reroutes to news_chef for further evaluation
    },
)

# Loop edges to allow re-evaluation if needed
workflow.add_edge("word_count_rewriter", "news_chef")
workflow.add_edge("current_club_researcher", "news_chef")
workflow.add_edge("market_value_researcher", "news_chef")

# Compile the workflow
app = workflow.compile()



In [ ]:
# Initial state with placeholders and a sample message
initial_state = {
    "messages": [HumanMessage(content="Messi will switch from FC Barcelona to Real Madrid in 2025")],
    "mentions_market_value": "no",
    "mentions_current_club": "no",
    "meets_100_words": "no"
}

result = app.invoke(initial_state)